
## Imports

In [ ]:
import pandas as pd
import numpy as np
import posixpath
from data_mining_project import preprocessing, data, OUTPUT_PATH

## Load Data

In [ ]:
file_name = "preprocessed_data.csv"  
filepath = posixpath.join(OUTPUT_PATH, file_name)
data_df = data.load_data_csv(filepath)

In [ ]:
data_df

In [ ]:
data_df = data_reformat_str_to_list(data_df, cols=["events_sequence", "seconds_to_incident_sequence", "dj_ac_state_sequence", "dj_dc_state_sequence"], col_type=int)
data_df = data_reformat_str_to_list(data_df, cols=["train_kph_sequence"], col_type=float)
data_df

In [ ]:
data_align = data_df[["events_sequence", "incident_type"]]

In [ ]:
data_align.head()

In [ ]:
data_align_use = data_align.copy()

In [ ]:
data_align_use

In [ ]:
data_align_use['events_sequence'] = data_align['events_sequence'].apply(lambda x: list(map(str, x)))

In [ ]:
data_align_use.head()

In [ ]:
data_align_use.at[0, 'events_sequence'][:10]

In [ ]:
unique_values_in_incendent_type = set(data_align_use['incident_type'].unique())

In [ ]:
unique_values_in_incendent_type

In [ ]:
data_align_incident_2 = data_align_use[data_align_use['incident_type'] == 2].copy().reset_index(drop=True)

In [ ]:
data_align_incident_2

In [ ]:
data_align_incident_2['events_sequence'][118][:10]

In [ ]:
len(data_align_incident_2)

In [ ]:
data_align_incident_3 = data_align_use[data_align_use['incident_type'] == 3].copy().reset_index(drop=True)

In [ ]:
len(data_align_incident_3)

In [ ]:
dico_for_alignment = dict()
for inc_type in unique_values_in_incendent_type:
    print(inc_type)
    data_align_per_incident = data_align_use[data_align_use['incident_type'] == inc_type].copy().reset_index(drop=True)
    print(data_align_per_incident.head())
    ls_event_list = []
    print(len(data_align_per_incident))
    for i in range(len(data_align_per_incident)):
        ls_event_list.append(data_align_per_incident['events_sequence'][i])
    dico_for_alignment[inc_type] = ls_event_list

In [ ]:
len(dico_for_alignment[3])

Code for local alignment

In [ ]:
# Implementing the Smith Waterman local alignment, changed a little using the paper: 
#Okada, D., Ino, F. & Hagihara, K. Accelerating the Smith-Waterman algorithm with interpair pruning and band optimization for the all-pairs comparison of base sequences. BMC Bioinformatics 16, 321 (2015). https://doi.org/10.1186/s12859-015-0744-4

import numpy as np

def smith_waterman(seq1, seq2): #Seq1 and seq2 are list of strings.
    #Gap penalty, match and mistmatch score
    #For simplicity, the gap score is constant for each gap of length 1 created (not affine as in the paper)
    #For simplicity, alpha is 1 and beta is -1. Creating a gap and having a mismatch have the same weight.
    gap_penalty = 0
    match = 1
    mismatch = -1
    
    # Generating the empty matrices for storing scores and tracing
    row = len(seq1) + 1 #first sequence in row => row[1] is 1st element of seq1
    col = len(seq2) + 1 #first sequence in column => col[1] is 1st element of seq2
    matrix_filling = np.zeros(shape=(row, col), dtype=np.int64)  
    matrix_tracing = np.zeros(shape=(row, col), dtype=np.int64)  
    
    # Initialising the variables to find the highest scoring cell
    max_score = -1
    max_index = (-1, -1)
    
    
    # Calculating the scores for all cells in the matrix
    for i in range(1, row):
        for j in range(1, col):
            # Calculating the diagonal score (match score)
            if seq1[i - 1] == seq2[j - 1]:
                S = match
            else:
                S = mismatch
                
            H = matrix_filling[i - 1, j - 1] + S
            
            # Calculating the vertical gap score
            F = matrix_filling[i - 1, j] + gap_penalty
            
            # Calculating the horizontal gap score
            E = matrix_filling[i, j - 1] + gap_penalty
            
            # Taking the highest score 
            matrix_filling[i, j] = max(0, H, F, E)
            
            # Tracking where the cell's value is coming from    
            if matrix_filling[i, j] == 0: 
                matrix_tracing[i, j] = 0
                
            elif matrix_filling[i, j] == E: 
                matrix_tracing[i, j] = 1 #meaning came from left
                
            elif matrix_filling[i, j] == F: 
                matrix_tracing[i, j] = 2 # meaning came from up
                
            elif matrix_filling[i, j] == H: 
                matrix_tracing[i, j] = 3 #meaning came from upper diagonal
                
            # Tracking the cell with the maximum score
            if matrix_filling[i, j] >= max_score:
                max_index = (i,j)
                max_score = matrix_filling[i, j]
    
    # Initialising the variables for tracing
    aligned_seq1 = []
    aligned_seq2 = [] 
    current_aligned_seq1 = ""  
    current_aligned_seq2 = "" 
    (max_i, max_j) = max_index #same the index of where the max score is (in the filling matrix)
    #print(max_score)
    #print(max_index)
    #print(matrix_filling)
    #print(matrix_tracing)
    
    # Tracing and computing the pathway with the local alignment
    while matrix_tracing[max_i, max_j] != 0:
        if matrix_tracing[max_i, max_j] == 3:
            current_aligned_seq1 = seq1[max_i - 1]
            current_aligned_seq2 = seq2[max_j - 1]
            max_i = max_i - 1
            max_j = max_j - 1
            
        elif matrix_tracing[max_i, max_j] == 2:
            current_aligned_seq1 = seq1[max_i - 1]
            current_aligned_seq2 = '-'
            max_i = max_i - 1    
            
        elif matrix_tracing[max_i, max_j] == 1:
            current_aligned_seq1 = '-'
            current_aligned_seq2 = seq2[max_j - 1]
            max_j = max_j - 1
            
        aligned_seq1.append(current_aligned_seq1)
        aligned_seq2.append(current_aligned_seq2)
    
    # Reversing the order of the sequences
    aligned_seq1.reverse()
    aligned_seq2.reverse()
    
    return aligned_seq1, aligned_seq2

In [ ]:
def sequence_in_common(seq1, seq2):
    if len(seq1) != len(seq2):
        print("The sequences are not the same size!!")
    else:
        seq_in_com = []
        for i in range(len(seq1)):
            if seq1[i]==seq2[i]:
                seq_in_com.append(seq1[i])
    return seq_in_com, len(seq_in_com)

In [ ]:
def seq_com_comparison(seq1, seq2):
    seq_a_1, seq_a_2 = smith_waterman(seq1, seq2)
    seq_com, score = sequence_in_common(seq_a_1, seq_a_2)
    return seq_com, score

In [ ]:
#Function for multiple sequence alignment
import math

#To be used within each incident class type
def multiple_sequence_alignment(list_seqs):
    ls_sorted_seqs = sorted(list_seqs, key=len) #sorts sequences to align by decreasing order: longuest sequence is ls_sorted_seqs[0]
    seq1 = ls_sorted_seqs.pop(0)
    seqs_score = math.inf
    for seq2 in ls_sorted_seqs:
        seq_com, score = seq_com_comparison(seq1, seq2)
        if score < seqs_score:
            seqs_score = score
        seq1 = seq_com
    return (seq_com, score) 

In [ ]:
multiple_sequence_alignment([list('abdjjjjjjjjjce'), list('abkkkkkkkkkkkkkkkkkkdee'), list('lllllllllabde')])

In [ ]:
for key in dico_for_alignment.keys():
    print(key, len(dico_for_alignment[key]))

In [ ]:
dico_seq_align_per_incident = dict()
for key in dico_for_alignment.keys():
    seq_com, score = multiple_sequence_alignment(dico_for_alignment[key])
    dico_seq_align_per_incident[key] = (seq_com, score)

In [ ]:
dico_seq_align_per_incident

In [ ]:
dico_seq_align_per_incident_2 = dict()
for key in dico_for_alignment.keys():
    seq_com, score = multiple_sequence_alignment(dico_for_alignment[key])
    dico_seq_align_per_incident_2[key] = (seq_com, score)

In [ ]:
dico_seq_align_per_incident_2

In [ ]:
dico_seq_align_per_incident_test = dict()
for key in dico_for_alignment.keys():
    seq_com, score = multiple_sequence_alignment(dico_for_alignment[key])
    dico_seq_align_per_incident_test[key] = (seq_com, score)
dico_seq_align_per_incident_test

In [ ]:
len(dico_seq_align_per_incident_test[3][0])

In [ ]:
for key in dico_seq_align_per_incident_test.keys():
    print(key)

1) Test if all ceq in common actually in the sequences! DONE
3) Test with a higher threshold: min number elements equal 15
2) Check if the sequence in common in or not sequences of other incident types!

In [ ]:
#To check that the algo works: the sequence in common is actually present in all sequences of the same incident type
import numpy as np

dico_actually_subsequence = dict()
for key in dico_seq_align_per_incident_test.keys():
    ls_in_com = dico_seq_align_per_incident_test[key][0]
    ls_seq_acc = dico_for_alignment[key]
    ls_in_bool = []
    for i in range(len(ls_seq_acc)):
        ls_in = np.isin(ls_in_com, ls_seq_acc[i])
        ls_in_bool.append(ls_in)
    bool_in = np.isin([False], ls_in_bool)
    if bool_in[0] == False:
        dico_actually_subsequence[key] = True
    else: 
        dico_actually_subsequence[key] = ls_in_bool

dico_actually_subsequence

In [ ]:
#Check if the common sequence of on idnicent type is present in other incident type
dict_occurences_seq_common_in_other_inc = dict()
for k in dico_seq_align_per_incident_test.keys():
    ls_com = dico_seq_align_per_incident_test[k][0]
    if ls_com == []:
        dict_occurences_seq_common_in_other_inc[k] = "This incident type doesn't have a common sequence"
    else:
        dico_seq_com_key_present_other_inc = dict()
        ls_incident_types = list(dico_seq_align_per_incident_test.keys())
        for inc_type in ls_incident_types:
            ls_seqs_other_inc = dico_for_alignment[inc_type] #list of sequences in incident type "inc_type"
            ls_in_bool = []
            for i in range(len(ls_seqs_other_inc)):
                ls_in = np.isin(ls_com, ls_seqs_other_inc[i]) #All true if ls_com_3 in ls_seqs_other_inc[i] OR with some false if some elements of ls_com_3 are in ls_seqs_other_inc[i]
                bool_in = np.isin([False], ls_in)#Returns False in only true in ls_in / returns True if a false is present in ls_in
                if bool_in[0] == False:
                    ls_in_bool.append(1) #1 => Seq_com_3 is present in the sequence
                else:
                    ls_in_bool.append(0) #0 => Seq_com_3 IS NOT present in the sequence
            sum_occ = sum(ls_in_bool)
            nb_seq = len(ls_in_bool)
            dico_seq_com_key_present_other_inc[inc_type] = (nb_seq, sum_occ)
        dict_occurences_seq_common_in_other_inc[k] = dico_seq_com_key_present_other_inc


#dico_seq_com_keu_present_other_inc[inc_type] = ls_in_bool
 #       dico_count_number_occ = dict()
  #      for key in dico_seq_com_3_present_other_inc.keys():
   #         ls = dico_seq_com_3_present_other_inc[key]
    #        sum_occ = sum(ls)
     #       nb_seq = len(ls)



In [ ]:
dict_occurences_seq_common_in_other_inc

In [ ]:
#Analysis for incident type 3:
ls_incidents = list(dict_occurences_seq_common_in_other_inc[3].keys())
ls_ratio = []
for inc_type in ls_incidents:
    nb, nb_occ = dict_occurences_seq_common_in_other_inc[3][inc_type]
    ls_ratio.append(nb_occ/nb)

import matplotlib.pyplot as plt

x_positions = np.arange(len(ls_incidents))

# Create the histogram
plt.figure(figsize=(8, 6))
plt.bar(x_positions, ls_ratio, color='skyblue', width=0.6, edgecolor='black')

plt.axhline(y=0.5, color='r')

plt.xticks(x_positions, ls_incidents)

# Add titles and labels
plt.title('Histogram of the ratio of the number of times the common sequence of incident type 3 (of length 25) \n appears in the event sequences of other incident types', fontsize=14)
plt.xlabel('Incident type', fontsize=12)
plt.ylabel('Ratios for each incident type', fontsize = 12)


# Display the plot
plt.show()

In [ ]:
#Analysis for incident type 6:
ls_incidents = list(dict_occurences_seq_common_in_other_inc[6].keys())
ls_ratio = []
for inc_type in ls_incidents:
    nb, nb_occ = dict_occurences_seq_common_in_other_inc[6][inc_type]
    ls_ratio.append(nb_occ/nb)

import matplotlib.pyplot as plt

x_positions = np.arange(len(ls_incidents))

# Create the histogram
plt.figure(figsize=(8, 6))
plt.bar(x_positions, ls_ratio, color='skyblue', width=0.6, edgecolor='black')

plt.axhline(y=0.5, color='r')

plt.xticks(x_positions, ls_incidents)

# Add titles and labels
plt.title('Histogram of the ratio of the number of times the common sequence of incident type 6 (of length 12) \n appears in the event sequences of other incident types', fontsize=14)
plt.xlabel('Incident type', fontsize=12)
plt.ylabel('Ratios for each incident type', fontsize = 12)


# Display the plot
plt.show()

In [ ]:
#Analysis for incident type 7:
ls_incidents = list(dict_occurences_seq_common_in_other_inc[7].keys())
ls_ratio = []
for inc_type in ls_incidents:
    nb, nb_occ = dict_occurences_seq_common_in_other_inc[7][inc_type]
    ls_ratio.append(nb_occ/nb)

import matplotlib.pyplot as plt

x_positions = np.arange(len(ls_incidents))

# Create the histogram
plt.figure(figsize=(8, 6))
plt.bar(x_positions, ls_ratio, color='skyblue', width=0.6, edgecolor='black')

plt.axhline(y=0.5, color='r')
plt.axhline(y=1, color='r')

plt.xticks(x_positions, ls_incidents)

# Add titles and labels
plt.title('Histogram of the ratio of the number of times the common sequence of incident type 7 (of length 7) \n appears in the event sequences of other incident types', fontsize=14)
plt.xlabel('Incident type', fontsize=12)
plt.ylabel('Ratios for each incident type', fontsize = 12)


# Display the plot
plt.show()

In [ ]:
#Analysis for incident type 11:
ls_incidents = list(dict_occurences_seq_common_in_other_inc[11].keys())
ls_ratio = []
for inc_type in ls_incidents:
    nb, nb_occ = dict_occurences_seq_common_in_other_inc[11][inc_type]
    ls_ratio.append(nb_occ/nb)

import matplotlib.pyplot as plt

x_positions = np.arange(len(ls_incidents))

# Create the histogram
plt.figure(figsize=(8, 6))
plt.bar(x_positions, ls_ratio, color='skyblue', width=0.6, edgecolor='black')

plt.axhline(y=0.5, color='r')
plt.axhline(y=1, color='r')

plt.xticks(x_positions, ls_incidents)

# Add titles and labels
plt.title('Histogram of the ratio of the number of times the common sequence of incident type 11 (of length 1) \n appears in the event sequences of other incident types', fontsize=14)
plt.xlabel('Incident type', fontsize=12)
plt.ylabel('Ratios for each incident type', fontsize = 12)


# Display the plot
plt.show()

In [ ]:
#Analysis for incident type 16:
ls_incidents = list(dict_occurences_seq_common_in_other_inc[16].keys())
ls_ratio = []
for inc_type in ls_incidents:
    nb, nb_occ = dict_occurences_seq_common_in_other_inc[16][inc_type]
    ls_ratio.append(nb_occ/nb)

import matplotlib.pyplot as plt

x_positions = np.arange(len(ls_incidents))

# Create the histogram
plt.figure(figsize=(8, 6))
plt.bar(x_positions, ls_ratio, color='skyblue', width=0.6, edgecolor='black')

plt.axhline(y=0.5, color='r')
plt.axhline(y=1, color='r')

plt.xticks(x_positions, ls_incidents)

# Add titles and labels
plt.title('Histogram of the ratio of the number of times the common sequence of incident type 16 (of length 3) \n appears in the event sequences of other incident types', fontsize=14)
plt.xlabel('Incident type', fontsize=12)
plt.ylabel('Ratios for each incident type', fontsize = 12)


# Display the plot
plt.show()

In [ ]:
#Analysis for incident type 17:
ls_incidents = list(dict_occurences_seq_common_in_other_inc[17].keys())
ls_ratio = []
for inc_type in ls_incidents:
    nb, nb_occ = dict_occurences_seq_common_in_other_inc[17][inc_type]
    ls_ratio.append(nb_occ/nb)

import matplotlib.pyplot as plt

x_positions = np.arange(len(ls_incidents))

# Create the histogram
plt.figure(figsize=(8, 6))
plt.bar(x_positions, ls_ratio, color='skyblue', width=0.6, edgecolor='black')

plt.axhline(y=0.5, color='r')
plt.axhline(y=1, color='r')

plt.xticks(x_positions, ls_incidents)

# Add titles and labels
plt.title('Histogram of the ratio of the number of times the common sequence of incident type 17 (of length 3) \n appears in the event sequences of other incident types', fontsize=14)
plt.xlabel('Incident type', fontsize=12)
plt.ylabel('Ratios for each incident type', fontsize = 12)


# Display the plot
plt.show()

In [ ]:
dico_len_seq_com = dict()
for inc_type in dico_seq_align_per_incident_test.keys():
    dico_len_seq_com[inc_type] = dico_seq_align_per_incident_test[inc_type][1]
dico_len_seq_com   

In [ ]:
dico_seq_align_per_incident_test[2][1]

Maybe remove elements that repeat everywhere

Check if each smaller sequence in the other events lists of the other classes